This notebook allow you to print results of models in tables

In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from twinews.utils import *
from twinews.evaluation.utils import *

In [3]:
if False:
    purgeSubsampledRankings()

In [4]:
def basicPrint(model, **kwargs):
    printReportKwargs = \
    {
        'model': model,
        'splitVersion': 2,
        'noSubsampling': True,
        'metaFilter': \
        {
            # "implementation": "gensim-lda",
        },
        'metricsFilter': {'ndcg'}, # None, {'ndcg', 'ndcg@10'}
        'sortBy': 'ndcg',
    }
    printReportKwargs = mergeDicts(printReportKwargs, kwargs)
    printReport(**printReportKwargs)

# Model lists per split version

In [ ]:
def printModelList(splitVersion):
    twinewsRankings = getTwinewsRankings()
    models = set()
    for key in twinewsRankings.keys():
        meta = twinewsRankings.getMeta(key)
        if meta["splitVersion"] == splitVersion:
            models.add(meta['model'])
    print(" ".join(sorted(list(models))) + " (" + str(len(models)) + ")")

In [ ]:
printModelList(2)

In [ ]:
printModelList(1)

# Reference models

In [ ]:
splitVersion = 2
df = printReport(model='ideal', splitVersion=splitVersion, noSubsampling=True, sortBy='ndcg')
df = printReport(model='random', splitVersion=splitVersion, noSubsampling=True, sortBy='ndcg')
df = printReport(model='worst', splitVersion=splitVersion, noSubsampling=True, sortBy='ndcg')

# Ad-hoc models

In [ ]:
basicPrint('bm25')

In [ ]:
basicPrint('BM25_yf', noSubsampling=False)

# Models that use vector representations

In [ ]:
basicPrint('lda')

In [ ]:
basicPrint('nmf')

In [ ]:
basicPrint('tfidf')

In [ ]:
basicPrint('dbert-ft')

In [ ]:
basicPrint('dbert-base')

In [ ]:
basicPrint('stylo')

In [ ]:
basicPrint('infersent')

In [ ]:
basicPrint('sent2vec')

In [ ]:
basicPrint('word2vec')

In [ ]:
basicPrint('doc2vec')

In [ ]:
basicPrint('usent')

In [ ]:
basicPrint('bert')

# Combinations of models

In [9]:
# Expected 182 rows:
basicPrint('combin')

These values are common to all rows 156:

	- maxUsers: None
	- splitVersion: 2
	- weights: [0.5, 0.5]
	- alphas: [0.5, 0.5]
	- betas: ['LOG', 'LOG']
	- model: combin


,id,models,rankAsScore,ndcg
61,combin-cf91b,"['bm25-1eb2a', 'dbert-ft-d1b5f']","[False, False]",0.58772
147,combin-f3aa7,"['bm25-1eb2a', 'doc2vec-19ca3']","[False, False]",0.57265
93,combin-a3995,"['dbert-ft-d1b5f', 'tfidf-6056f']","[False, False]",0.57127
4,combin-10a53,"['bm25-1eb2a', 'tfidf-6056f']","[True, True]",0.56602
129,combin-26bf3,"['bm25-1eb2a', 'tfidf-6056f']","[False, False]",0.565
47,combin-6b411,"['dbert-ft-d1b5f', 'doc2vec-19ca3']","[False, False]",0.56448
69,combin-a9c56,"['bm25-1eb2a', 'doc2vec-19ca3']","[True, True]",0.56408
96,combin-1e53b,"['bm25-1eb2a', 'dbert-base-aafd1']","[False, False]",0.5598
68,combin-7d8fe,"['bm25-1eb2a', 'word2vec-4a2c5']","[False, False]",0.55949
14,combin-a7b1a,"['bm25-1eb2a', 'dbert-ft-d1b5f']","[True, True]",0.55912


# Deep Learning models

# Reports on the test set (split version 1)

In [5]:
splitVersion = 1

In [6]:
report = printReport(splitVersion=splitVersion, model='ideal')
report = printReport(splitVersion=splitVersion, model='random')
report = printReport(splitVersion=splitVersion, model='worst')

,id,maxUsers,model,splitVersion,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100
0,ideal-5eb2b,None,ideal,1,1,1,1,1,1,0.67484,0.1063


These values are common to all rows 10:

	- maxUsers: None
	- splitVersion: 1
	- model: random


,id,seed,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100
8,random-15c7e,4,0.0146,0.02233,0.25101,0.00103,0.0575,0.00156,0.0164
1,random-a039c,3,0.01305,0.0088,0.24404,1e-05,0.00206,2e-05,0.00148
2,random-ab01e,6,0.01294,0.0388,0.24158,0.02029,0.01828,0.02917,0.00596
9,random-03419,8,0.01278,0.01932,0.24411,0.0076,0.00772,0.01179,0.00342
6,random-8217a,2,0.01218,0.0113,0.24015,0.00013,0.00276,0.00012,0.00159
4,random-35ee6,7,0.01209,0.01815,0.2383,0,0.01147,0,0.00383
7,random-8e615,1,0.01166,0.01222,0.23957,0.00127,0.03055,0.00192,0.01023
0,random-bf97b,0,0.01139,0.023,0.23543,5e-05,0.02322,7e-05,0.00656
3,random-fbe0a,5,0.011,0.0119,0.2342,0.00143,0.02565,0.00198,0.01001
5,random-8373e,9,0.0101,0.0066,0.23379,1e-05,0.00072,1e-05,0.00064


,id,maxUsers,model,splitVersion,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100
0,worst-559a0,None,worst,1,0.00587,0.001,0.20599,0,0,0,0


In [7]:
report = printReport\
(
    splitVersion=splitVersion,
    excludedModels={'combin', 'ideal', 'worst', 'random'},
    # metricsFilter={'ndcg'},
    sortBy='ndcg',
    noSubsampling=True,
    onlyFields={'id', 'model'}
)

,id,model,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100
2,bm25-933f7,bm25,0.36316,0.61723,0.60819,0.44273,0.53604,0.31301,0.07674
4,doc2vec-e013a,doc2vec,0.3232,0.58795,0.58202,0.40184,0.50388,0.28369,0.07424
5,tfidf-b0e81,tfidf,0.32828,0.56854,0.57847,0.40164,0.49896,0.28549,0.07362
8,nmf-6078e,nmf,0.26979,0.51627,0.54098,0.33832,0.4599,0.24526,0.07211
1,dbert-ft-7847a,dbert-ft,0.27155,0.53054,0.53664,0.3472,0.44411,0.24932,0.06864
6,usent-64ec0,usent,0.24835,0.49151,0.5172,0.31744,0.41832,0.22694,0.06454
10,lda-82272,lda,0.19536,0.40872,0.47486,0.24493,0.37615,0.18029,0.06437
12,dbert-base-d092a,dbert-base,0.18273,0.43752,0.46352,0.25297,0.34227,0.17994,0.05452
9,infersent-77ec7,infersent,0.18162,0.42212,0.46136,0.246,0.34086,0.17558,0.05449
0,bert-a7c9a,bert,0.152,0.37992,0.43154,0.21174,0.29781,0.15102,0.04855


In [16]:
# Expected 156 rows:
report = printReport\
(
    splitVersion=splitVersion,
    model='combin',
    # metricsFilter={'ndcg'},
    sortBy='ndcg',
    noSubsampling=True,
)

These values are common to all rows 156:

	- maxUsers: None
	- splitVersion: 1
	- weights: [0.5, 0.5]
	- alphas: [0.5, 0.5]
	- betas: ['LOG', 'LOG']
	- model: combin


,id,models,rankAsScore,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100
45,combin-8f846,"['bm25-933f7', 'dbert-ft-7847a']","[False, False]",0.39517,0.66123,0.63565,0.47918,0.57028,0.3363,0.07999
130,combin-6fad8,"['bm25-933f7', 'doc2vec-e013a']","[False, False]",0.37663,0.63976,0.62146,0.45809,0.55266,0.32122,0.07817
102,combin-1c882,"['dbert-ft-7847a', 'tfidf-b0e81']","[False, False]",0.37015,0.63285,0.6155,0.45244,0.54458,0.3185,0.07783
115,combin-6631d,"['bm25-933f7', 'tfidf-b0e81']","[True, True]",0.36947,0.63546,0.61475,0.45295,0.54231,0.31752,0.07651
81,combin-1c964,"['bm25-933f7', 'doc2vec-e013a']","[True, True]",0.36506,0.6335,0.61391,0.44685,0.54347,0.31323,0.07748
12,combin-66946,"['bm25-933f7', 'tfidf-b0e81']","[False, False]",0.36758,0.63049,0.61267,0.44966,0.53993,0.31542,0.0764
86,combin-d0fd0,"['dbert-ft-7847a', 'doc2vec-e013a']","[False, False]",0.3584,0.62854,0.61009,0.44044,0.54034,0.31016,0.07831
34,combin-904f0,"['bm25-933f7', 'word2vec-f3c38']","[False, False]",0.36316,0.61723,0.60819,0.44273,0.53604,0.31301,0.07674
17,combin-50f90,"['bm25-933f7', 'dbert-base-d092a']","[False, False]",0.36089,0.6235,0.60802,0.44445,0.53385,0.31292,0.07555
128,combin-959fb,"['bm25-933f7', 'nmf-6078e']","[False, False]",0.35361,0.62588,0.60763,0.43587,0.53769,0.30533,0.07742
